In [4]:
import pandas as pd
import numpy as np
import fastf1 as ff1
from datetime import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
# get the schedule df's by year

def get_dataframe_schedule(year):
    df = ff1.get_event_schedule(year)
    return (
        df
        .drop(columns=["Location", "OfficialEventName", "Session1Date", "Session1DateUtc", "Session2Date", "Session2DateUtc", "Session3Date", "Session3DateUtc",
                       "Session4Date", "Session4DateUtc", "Session5Date", "Session5DateUtc", "F1ApiSupport"])
        .loc[~df['EventName'].str.contains('Pre-Season', na=False)]
        .reset_index(drop=True)
    )

In [3]:
df1 = get_dataframe_schedule(2024)
df1

req         WARNING 	DEFAULT CACHE ENABLED! (4.5 GB) C:\Users\Caio\AppData\Local\Temp\fastf1


,RoundNumber,Country,EventDate,EventName,EventFormat,Session1,Session2,Session3,Session4,Session5
0,1,Bahrain,2024-03-02,Bahrain Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
1,2,Saudi Arabia,2024-03-09,Saudi Arabian Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
2,3,Australia,2024-03-24,Australian Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
3,4,Japan,2024-04-07,Japanese Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
4,5,China,2024-04-21,Chinese Grand Prix,sprint_qualifying,Practice 1,Sprint Qualifying,Sprint,Qualifying,Race
5,6,United States,2024-05-05,Miami Grand Prix,sprint_qualifying,Practice 1,Sprint Qualifying,Sprint,Qualifying,Race
6,7,Italy,2024-05-19,Emilia Romagna Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
7,8,Monaco,2024-05-26,Monaco Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
8,9,Canada,2024-06-09,Canadian Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race
9,10,Spain,2024-06-23,Spanish Grand Prix,conventional,Practice 1,Practice 2,Practice 3,Qualifying,Race


In [4]:
# get the race results df's

def get_race(year, gp):
    race = ff1.get_session(year, gp, "R")
    race.load(laps=False, telemetry=False, weather=False, messages=False, livedata=False)
    race_results_df = race.results
    race_results_df = race_results_df.drop(columns=["BroadcastName", "Abbreviation", "DriverId", "TeamColor", "TeamId", "FirstName", "LastName", "HeadshotUrl", "CountryCode", "Q1", "Q2", "Q3", "Time"])
    df_event = get_dataframe_schedule(year)
    event_name = df_event.at[gp-1, "EventName"]
    race_results_df["EventName"] = event_name
    return race_results_df

In [ ]:
# concat the results df's

race_dataframes = []
for race in range(1, df1["RoundNumber"].max() + 1):
    df_race = get_race(2023, race)
    race_dataframes.append(df_race)

df_race_concat = pd.concat(race_dataframes, axis=0, ignore_index=True)

In [5]:
# get laps data from race

def get_laps_data(year, gp):
    race = ff1.get_session(year, gp, "R")
    race.load(telemetry=False, weather=False, messages=False, livedata=False)
    df_laps = race.laps
    df = df_laps.drop(columns=["Time", "PitOutTime", "PitInTime", "Sector1SessionTime", "Sector2SessionTime", "Sector3SessionTime", "SpeedFL", "SpeedST",
                           "IsPersonalBest", "LapStartTime", "LapStartDate", "TrackStatus", "DeletedReason", "FastF1Generated", "IsAccurate"])
    return df

In [ ]:
laps_dataframe = []
for race in range(1, df1["RoundNumber"].max() + 1):
    lap = get_laps_data(2023, race)
    laps_dataframe.append(lap)


laps_df_concat = pd.concat(laps_dataframe, axis=0, ignore_index=True)

In [6]:
season = dt.now().year
events = ff1.get_event_schedule(season)
today = pd.Timestamp.now()
past_events = events[events["EventDate"] <= today]

In [7]:
past_events

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,Session2,Session2Date,Session2DateUtc,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,2024-02-23,Pre-Season Testing,testing,Practice 1,2024-02-21 10:00:00+03:00,2024-02-21 07:00:00,Practice 2,2024-02-22 10:00:00+03:00,2024-02-22 07:00:00,Practice 3,2024-02-23 10:00:00+03:00,2024-02-23 07:00:00,None,NaT,NaT,None,NaT,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,2024-03-09,Saudi Arabian Grand Prix,conventional,Practice 1,2024-03-07 16:30:00+03:00,2024-03-07 13:30:00,Practice 2,2024-03-07 20:10:00+03:00,2024-03-07 17:10:00,Practice 3,2024-03-08 16:30:00+03:00,2024-03-08 13:30:00,Qualifying,2024-03-08 20:00:00+03:00,2024-03-08 17:00:00,Race,2024-03-09 20:00:00+03:00,2024-03-09 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,2024-03-24,Australian Grand Prix,conventional,Practice 1,2024-03-22 12:30:00+11:00,2024-03-22 01:30:00,Practice 2,2024-03-22 16:00:00+11:00,2024-03-22 05:00:00,Practice 3,2024-03-23 12:30:00+11:00,2024-03-23 01:30:00,Qualifying,2024-03-23 16:00:00+11:00,2024-03-23 05:00:00,Race,2024-03-24 15:00:00+11:00,2024-03-24 04:00:00,True
4,4,Japan,Suzuka,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,2024-04-07,Japanese Grand Prix,conventional,Practice 1,2024-04-05 11:30:00+09:00,2024-04-05 02:30:00,Practice 2,2024-04-05 15:00:00+09:00,2024-04-05 06:00:00,Practice 3,2024-04-06 11:30:00+09:00,2024-04-06 02:30:00,Qualifying,2024-04-06 15:00:00+09:00,2024-04-06 06:00:00,Race,2024-04-07 14:00:00+09:00,2024-04-07 05:00:00,True
5,5,China,Shanghai,FORMULA 1 LENOVO CHINESE GRAND PRIX 2024,2024-04-21,Chinese Grand Prix,sprint_qualifying,Practice 1,2024-04-19 11:30:00+08:00,2024-04-19 03:30:00,Sprint Qualifying,2024-04-19 15:30:00+08:00,2024-04-19 07:30:00,Sprint,2024-04-20 11:00:00+08:00,2024-04-20 03:00:00,Qualifying,2024-04-20 15:00:00+08:00,2024-04-20 07:00:00,Race,2024-04-21 15:00:00+08:00,2024-04-21 07:00:00,True
6,6,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2024,2024-05-05,Miami Grand Prix,sprint_qualifying,Practice 1,2024-05-03 12:30:00-04:00,2024-05-03 16:30:00,Sprint Qualifying,2024-05-03 16:30:00-04:00,2024-05-03 20:30:00,Sprint,2024-05-04 12:00:00-04:00,2024-05-04 16:00:00,Qualifying,2024-05-04 16:00:00-04:00,2024-05-04 20:00:00,Race,2024-05-05 16:00:00-04:00,2024-05-05 20:00:00,True
7,7,Italy,Imola,FORMULA 1 MSC CRUISES GRAN PREMIO DEL MADE IN ITALY E DELL'EMILIA-ROMAGNA 2024,2024-05-19,Emilia Romagna Grand Prix,conventional,Practice 1,2024-05-17 13:30:00+02:00,2024-05-17 11:30:00,Practice 2,2024-05-17 17:00:00+02:00,2024-05-17 15:00:00,Practice 3,2024-05-18 12:30:00+02:00,2024-05-18 10:30:00,Qualifying,2024-05-18 16:00:00+02:00,2024-05-18 14:00:00,Race,2024-05-19 15:00:00+02:00,2024-05-19 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2024,2024-05-26,Monaco Grand Prix,conventional,Practice 1,2024-05-24 13:30:00+02:00,2024-05-24 11:30:00,Practice 2,2024-05-24 17:00:00+02:00,2024-05-24 15:00:00,Practice 3,2024-05-25 12:30:00+02:00,2024-05-25 10:30:00,Qualifying,2024-05-25 16:00:00+02:00,2024-05-25 14:00:00,Race,2024-05-26 15:00:00+02:00,2024-05-26 13:00:00,True
9,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2024,2024-06-09,Canadian Grand Prix,conventional,Practice 1,2024-06-07 13:30:00-04:00,2024-06-07 17:30:00,Practice 2,2024-06-07 17:00:00-04:00,2024-06-07 21:00:00,Practice 3,2024-

In [ ]:
race_dataframes = []
laps_dataframe = []
for race in range(1, past_events["RoundNumber"].max() + 1):
    df_race = get_race(2024, race)
    lap = get_laps_data(2024, race)

    if pd.isna(df_race["Position"].iloc[0]):
        max_lap_number = lap['LapNumber'].max()
        last_lap = lap[lap["LapNumber"] == max_lap_number]
        last_positions = last_lap.set_index('DriverNumber')['Position']
        df_race["Position"] = df_race["DriverNumber"].map(last_positions).fillna(0).astype(int)
        df_race['IsZero'] = df_race['Position'] == 0
        df_race = df_race.sort_values(by=['IsZero', 'Position']).reset_index(drop=True)
        df_race = df_race.drop(columns=['IsZero'])

    race_dataframes.append(df_race)
    laps_dataframe.append(lap)

df_race_concat = pd.concat(race_dataframes, axis=0, ignore_index=True)
laps_df_concat = pd.concat(laps_dataframe, axis=0, ignore_index=True)

df_merge_schedule_results = pd.merge(past_events, df_race_concat, on="EventName")
df_merge_all = pd.merge(df_merge_schedule_results, laps_df_concat, on="DriverNumber")

In [ ]:
df_merge_all

In [ ]:
df_merge_all.to_csv("race_results_laps_2024.csv")

In [61]:
df = pd.read_csv("race_results_laps_2024.csv")

C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\1188868395.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("race_results_laps_2024.csv")


In [62]:
df.head()

,Unnamed: 0,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,Session2,Session2Date,Session2DateUtc,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport,DriverNumber,TeamName,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Team,Position_y,Deleted
0,0,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:37.284000,1.0,1.0,NaN,0 days 00:00:41.266000,0 days 00:00:23.616000,234.0,250.0,SOFT,4.0,False,Red Bull Racing,1.0,NaN
1,1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:36.296000,2.0,1.0,0 days 00:00:30.916000,0 days 00:00:41.661000,0 days 00:00:23.719000,232.0,248.0,SOFT,5.0,False,Red Bull Racing,1.0,NaN
2,2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:36.753000,3.0,1.0,0 days 00:00:30.999000,0 days 00:00:41.966000,0 days 00:00:23.788000,231.0,243.0,SOFT,6.0,False,Red Bull Racing,1.0,NaN
3,3,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:36.647000,4.0,1.0,0 days 00:00:30.931000,0 days 00:00:41.892000,0 days 00:00:23.824000,233.0,253.0,SOFT,7.0,False,Red Bull Racing,1.0,NaN
4,4,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:37.173000,5.0,1.0,0 days 00:00:31.255000,0 days 00:00:42.056000,0 days 00:00:23.862000,231.0,245.0,SOFT,8.0,False,Red Bull Racing,1.0,NaN


In [63]:
df.isnull().sum()

Unnamed: 0                 0
RoundNumber                0
Country                    0
Location                   0
OfficialEventName          0
EventDate                  0
EventName                  0
EventFormat                0
Session1                   0
Session1Date               0
Session1DateUtc            0
Session2                   0
Session2Date               0
Session2DateUtc            0
Session3                   0
Session3Date               0
Session3DateUtc            0
Session4                   0
Session4Date               0
Session4DateUtc            0
Session5                   0
Session5Date               0
Session5DateUtc            0
F1ApiSupport               0
DriverNumber               0
TeamName                   0
FullName                   0
Position_x                 0
ClassifiedPosition     29612
GridPosition           29612
Status                 29612
Points                 29612
Driver                     0
LapTime                 1949
LapNumber     

In [64]:
# mean speed

def fill_na_speed(df, cols):
    for col in cols:
        mean_values = df.groupby(["EventName", "FullName"])[col].transform('mean')
        df[col].fillna(mean_values, inplace=True)
    return df

In [65]:
columns = ["SpeedI1", "SpeedI2"]

In [66]:
df = fill_na_speed(df, columns)

C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\1651287617.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_values, inplace=True)
C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\1651287617.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

In [67]:
df.head(6)

,Unnamed: 0,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,Session2,Session2Date,Session2DateUtc,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport,DriverNumber,TeamName,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Team,Position_y,Deleted
0,0,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:37.284000,1.0,1.0,NaN,0 days 00:00:41.266000,0 days 00:00:23.616000,234.000000,250.0,SOFT,4.0,False,Red Bull Racing,1.0,NaN
1,1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:36.296000,2.0,1.0,0 days 00:00:30.916000,0 days 00:00:41.661000,0 days 00:00:23.719000,232.000000,248.0,SOFT,5.0,False,Red Bull Racing,1.0,NaN
2,2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:36.753000,3.0,1.0,0 days 00:00:30.999000,0 days 00:00:41.966000,0 days 00:00:23.788000,231.000000,243.0,SOFT,6.0,False,Red Bull Racing,1.0,NaN
3,3,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:36.647000,4.0,1.0,0 days 00:00:30.931000,0 days 00:00:41.892000,0 days 00:00:23.824000,233.000000,253.0,SOFT,7.0,False,Red Bull Racing,1.0,NaN
4,4,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,0 days 00:01:37.173000,5.0,1.0,0 days 00:00:31.255000,0 days 00:00:42.056000,0 days 00:00:23.862000,231.000000,245.0,SOFT,8.0,False,Red Bull Racing,1.0,NaN
5,5,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing

In [68]:
df[["SpeedI1", "SpeedI2"]].isna().sum()

SpeedI1    0
SpeedI2    0
dtype: int64

In [69]:
df["Sector1Time"] = pd.to_timedelta(df["Sector1Time"])
df["Sector2Time"] = pd.to_timedelta(df["Sector2Time"])
df["Sector3Time"] = pd.to_timedelta(df["Sector3Time"])

In [70]:
def fill_na_sectorTime(df, cols):
    for col in cols:
        mean_values = df.groupby(["EventName", "FullName"])[col].transform(lambda x: x.dropna().mean())
        df[col].fillna(mean_values, inplace=True)
    return df

In [71]:
columns_sectorTime = ['Sector1Time', 'Sector2Time', 'Sector3Time']

In [72]:
df = fill_na_sectorTime(df, columns_sectorTime)

C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\2997792398.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_values, inplace=True)
C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\2997792398.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

In [73]:
df[['Sector1Time', 'Sector2Time', 'Sector3Time']].isna().sum()

Sector1Time    0
Sector2Time    0
Sector3Time    0
dtype: int64

In [74]:
df.isna().sum()

Unnamed: 0                 0
RoundNumber                0
Country                    0
Location                   0
OfficialEventName          0
EventDate                  0
EventName                  0
EventFormat                0
Session1                   0
Session1Date               0
Session1DateUtc            0
Session2                   0
Session2Date               0
Session2DateUtc            0
Session3                   0
Session3Date               0
Session3DateUtc            0
Session4                   0
Session4Date               0
Session4DateUtc            0
Session5                   0
Session5Date               0
Session5DateUtc            0
F1ApiSupport               0
DriverNumber               0
TeamName                   0
FullName                   0
Position_x                 0
ClassifiedPosition     29612
GridPosition           29612
Status                 29612
Points                 29612
Driver                     0
LapTime                 1949
LapNumber     

In [75]:
df[df["LapTime"].isnull()]

,Unnamed: 0,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,Session2,Session2Date,Session2DateUtc,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport,DriverNumber,TeamName,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,Driver,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Team,Position_y,Deleted
64,64,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,NaN,8.0,2.0,0 days 00:01:13.947000,0 days 00:00:57.475000,0 days 00:00:57.661000,158.000000,96.00000,HARD,1.0,True,Red Bull Racing,2.0,NaN
65,65,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,NaN,9.0,2.0,0 days 00:00:56.642000,0 days 00:00:49.042000,0 days 00:00:55.311000,183.000000,131.00000,HARD,2.0,True,Red Bull Racing,2.0,NaN
110,110,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,NaN,4.0,1.0,0 days 00:00:28.756000,0 days 00:00:18.353000,0 days 00:00:27.164187919,270.000000,308.00000,MEDIUM,4.0,True,Red Bull Racing,19.0,NaN
112,112,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,NaN,2.0,2.0,0 days 00:00:27.778819672,0 days 00:00:56.617000,0 days 00:00:39.032000,179.000000,245.00000,MEDIUM,2.0,False,Red Bull Racing,1.0,NaN
113,113,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02 00:00:00,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,Practice 2,2024-02-29 18:00:00+03:00,2024-02-29 15:00:00,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True,1,Red Bull Racing,Max Verstappen,1.0,1,1.0,Finished,26.0,VER,NaN,3.0,2.0,0 days 00:00:27.778819672,0 days 00:00:42.768000,0 days 00:00:18.603000,279.000000,291.00000,MEDIUM,3.0,False,Red Bull Racing,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190225,190225,13,Hungary,Budapest,FORMULA 1 HUNGARIAN GRAND PRIX 2024,2024-07-21 00:00:00,Hungarian Grand Prix,conventional,Practice 1,2024-07-19 13:30:00+02:00,2024-07-19 11:30:00,Practice 2,2024-07-19 17:00:00+02:00,2024-07-19 15:00:00,Practice 3,2024

In [76]:
def fill_na_laptime(df, cols):
    for col in cols:
        mean_values = df.groupby(["EventName", "FullName"])[col].transform(lambda x: x.dropna().mean())
        df[col].fillna(mean_values, inplace=True)
    return df

In [77]:
df["LapTime"] = pd.to_timedelta(df["LapTime"])

In [78]:
laptime_column = ["LapTime"]

In [79]:
df = fill_na_laptime(df, laptime_column)

C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\3126317024.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_values, inplace=True)


In [80]:
df.isna().sum()

Unnamed: 0                 0
RoundNumber                0
Country                    0
Location                   0
OfficialEventName          0
EventDate                  0
EventName                  0
EventFormat                0
Session1                   0
Session1Date               0
Session1DateUtc            0
Session2                   0
Session2Date               0
Session2DateUtc            0
Session3                   0
Session3Date               0
Session3DateUtc            0
Session4                   0
Session4Date               0
Session4DateUtc            0
Session5                   0
Session5Date               0
Session5DateUtc            0
F1ApiSupport               0
DriverNumber               0
TeamName                   0
FullName                   0
Position_x                 0
ClassifiedPosition     29612
GridPosition           29612
Status                 29612
Points                 29612
Driver                     0
LapTime                    0
LapNumber     

In [81]:
df = df.drop(columns=["Unnamed: 0", "RoundNumber", "Country", "Location", "OfficialEventName", "EventDate", "EventFormat", "Session1", "Session1Date", "Session1DateUtc",
                                    "Session2", "Session2Date", "Session2DateUtc", "Session3", "Session3Date", "Session3DateUtc", "Session4", "Session4Date", "Session4DateUtc", "Session5", "Session5Date", "F1ApiSupport",
                                    "DriverNumber", "TeamName", "Driver", "Team", "Deleted"])

In [82]:
df

,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y
0,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,Finished,26.0,0 days 00:01:37.284000,1.0,1.0,0 days 00:00:27.778819672,0 days 00:00:41.266000,0 days 00:00:23.616000,234.000000,250.0,SOFT,4.0,False,1.0
1,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,Finished,26.0,0 days 00:01:36.296000,2.0,1.0,0 days 00:00:30.916000,0 days 00:00:41.661000,0 days 00:00:23.719000,232.000000,248.0,SOFT,5.0,False,1.0
2,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,Finished,26.0,0 days 00:01:36.753000,3.0,1.0,0 days 00:00:30.999000,0 days 00:00:41.966000,0 days 00:00:23.788000,231.000000,243.0,SOFT,6.0,False,1.0
3,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,Finished,26.0,0 days 00:01:36.647000,4.0,1.0,0 days 00:00:30.931000,0 days 00:00:41.892000,0 days 00:00:23.824000,233.000000,253.0,SOFT,7.0,False,1.0
4,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,Finished,26.0,0 days 00:01:37.173000,5.0,1.0,0 days 00:00:31.255000,0 days 00:00:42.056000,0 days 00:00:23.862000,231.000000,245.0,SOFT,8.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191133,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,R,0.0,Hydraulics,0.0,0 days 00:01:41.671000,29.0,2.0,0 days 00:00:47.902000,0 days 00:00:30.206000,0 days 00:00:23.563000,279.000000,235.0,MEDIUM,1.0,True,19.0
191134,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,R,0.0,Hydraulics,0.0,0 days 00:01:23.446000,30.0,2.0,0 days 00:00:29.727000,0 days 00:00:30.081000,0 days 00:00:23.638000,276.000000,237.0,MEDIUM,2.0,True,17.0
191135,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,R,0.0,Hydraulics,0.0,0 days 00:01:23.340000,31.0,2.0,0 days 00:00:29.754000,0 days 00:00:30.046000,0 days 00:00:23.540000,277.000000,240.0,MEDIUM,3.0,True,16.0
191136,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,R,0.0,Hydraulics,0.0,0 days 00:01:23.484000,32.0,2.0,0 days 00:00:29.652000,0 days 00:00:30.176000,0 days 00:00:23.656000,241.721925,241.0,MEDIUM,4.0,True,15.0


In [83]:
cols_to_fill = ["ClassifiedPosition", "GridPosition", "Points"]

for col in cols_to_fill:
    df.loc[df["Status"] != "Finished", col] = df.loc[df["Status"] != "Finished", col].fillna(0)

In [84]:
df.isnull().sum()

EventName                 0
Session5DateUtc           0
FullName                  0
Position_x                0
ClassifiedPosition        0
GridPosition              0
Status                29612
Points                    0
LapTime                   0
LapNumber                 0
Stint                     0
Sector1Time               0
Sector2Time               0
Sector3Time               0
SpeedI1                   0
SpeedI2                   0
Compound                  0
TyreLife                  0
FreshTyre                 0
Position_y              102
dtype: int64

In [85]:
df["Status"] = np.where(df["Status"] == "Finished", 1, 0)

In [86]:
df.ClassifiedPosition.unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', 'R', '1', '2', '19', 0, 'W'],
      dtype=object)

In [87]:
values_to_replace = ["R", "D", "E", "W", "F", "N"]

In [88]:
df["ClassifiedPosition"] = np.where(df["ClassifiedPosition"].isin(values_to_replace), 0, df["ClassifiedPosition"])

In [89]:
df

,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y
0,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0 days 00:01:37.284000,1.0,1.0,0 days 00:00:27.778819672,0 days 00:00:41.266000,0 days 00:00:23.616000,234.000000,250.0,SOFT,4.0,False,1.0
1,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0 days 00:01:36.296000,2.0,1.0,0 days 00:00:30.916000,0 days 00:00:41.661000,0 days 00:00:23.719000,232.000000,248.0,SOFT,5.0,False,1.0
2,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0 days 00:01:36.753000,3.0,1.0,0 days 00:00:30.999000,0 days 00:00:41.966000,0 days 00:00:23.788000,231.000000,243.0,SOFT,6.0,False,1.0
3,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0 days 00:01:36.647000,4.0,1.0,0 days 00:00:30.931000,0 days 00:00:41.892000,0 days 00:00:23.824000,233.000000,253.0,SOFT,7.0,False,1.0
4,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0 days 00:01:37.173000,5.0,1.0,0 days 00:00:31.255000,0 days 00:00:42.056000,0 days 00:00:23.862000,231.000000,245.0,SOFT,8.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191133,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,0 days 00:01:41.671000,29.0,2.0,0 days 00:00:47.902000,0 days 00:00:30.206000,0 days 00:00:23.563000,279.000000,235.0,MEDIUM,1.0,True,19.0
191134,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,0 days 00:01:23.446000,30.0,2.0,0 days 00:00:29.727000,0 days 00:00:30.081000,0 days 00:00:23.638000,276.000000,237.0,MEDIUM,2.0,True,17.0
191135,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,0 days 00:01:23.340000,31.0,2.0,0 days 00:00:29.754000,0 days 00:00:30.046000,0 days 00:00:23.540000,277.000000,240.0,MEDIUM,3.0,True,16.0
191136,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,0 days 00:01:23.484000,32.0,2.0,0 days 00:00:29.652000,0 days 00:00:30.176000,0 days 00:00:23.656000,241.721925,241.0,MEDIUM,4.0,True,15.0


In [90]:
df['Sector1Time'] = df['Sector1Time'].dt.total_seconds()
df['Sector2Time'] = df['Sector2Time'].dt.total_seconds()
df['Sector3Time'] = df['Sector3Time'].dt.total_seconds()

In [91]:
df['LapTime'] = df['LapTime'].dt.total_seconds()

In [92]:
df

,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y
0,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,97.284,1.0,1.0,27.77882,41.266,23.616,234.000000,250.0,SOFT,4.0,False,1.0
1,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,96.296,2.0,1.0,30.91600,41.661,23.719,232.000000,248.0,SOFT,5.0,False,1.0
2,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,96.753,3.0,1.0,30.99900,41.966,23.788,231.000000,243.0,SOFT,6.0,False,1.0
3,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,96.647,4.0,1.0,30.93100,41.892,23.824,233.000000,253.0,SOFT,7.0,False,1.0
4,Bahrain Grand Prix,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,97.173,5.0,1.0,31.25500,42.056,23.862,231.000000,245.0,SOFT,8.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191133,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,101.671,29.0,2.0,47.90200,30.206,23.563,279.000000,235.0,MEDIUM,1.0,True,19.0
191134,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,83.446,30.0,2.0,29.72700,30.081,23.638,276.000000,237.0,MEDIUM,2.0,True,17.0
191135,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,83.340,31.0,2.0,29.75400,30.046,23.540,277.000000,240.0,MEDIUM,3.0,True,16.0
191136,Hungarian Grand Prix,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,83.484,32.0,2.0,29.65200,30.176,23.656,241.721925,241.0,MEDIUM,4.0,True,15.0


In [93]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

categorical_columns = ["EventName", "Compound"]

for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

In [94]:
df["NameEncoder"] = label_encoder.fit_transform(df["FullName"])

In [96]:
encoder_combination = df[["FullName", "NameEncoder"]].drop_duplicates()

In [97]:
encoder_combination

,FullName,NameEncoder
0,Max Verstappen,13
747,Sergio Perez,18
1451,Carlos Sainz,1
2185,Charles Leclerc,2
2956,George Russell,6
3737,Lando Norris,10
4531,Lewis Hamilton,11
5290,Oscar Piastri,16
6091,Fernando Alonso,5
6887,Lance Stroll,9


In [98]:
df["FreshTyre"] = df["FreshTyre"].astype(int)

In [99]:
df

,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y,NameEncoder
0,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,97.284,1.0,1.0,27.77882,41.266,23.616,234.000000,250.0,3,4.0,0,1.0,13
1,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,96.296,2.0,1.0,30.91600,41.661,23.719,232.000000,248.0,3,5.0,0,1.0,13
2,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,96.753,3.0,1.0,30.99900,41.966,23.788,231.000000,243.0,3,6.0,0,1.0,13
3,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,96.647,4.0,1.0,30.93100,41.892,23.824,233.000000,253.0,3,7.0,0,1.0,13
4,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,97.173,5.0,1.0,31.25500,42.056,23.862,231.000000,245.0,3,8.0,0,1.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191133,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,101.671,29.0,2.0,47.90200,30.206,23.563,279.000000,235.0,2,1.0,1,19.0,17
191134,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,83.446,30.0,2.0,29.72700,30.081,23.638,276.000000,237.0,2,2.0,1,17.0,17
191135,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,83.340,31.0,2.0,29.75400,30.046,23.540,277.000000,240.0,2,3.0,1,16.0,17
191136,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,83.484,32.0,2.0,29.65200,30.176,23.656,241.721925,241.0,2,4.0,1,15.0,17


In [100]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df[['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'TyreLife']] = scaler.fit_transform(
    df[['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'TyreLife']]
)

In [101]:
df = df.dropna()

In [102]:
df.isnull().sum()

EventName             0
Session5DateUtc       0
FullName              0
Position_x            0
ClassifiedPosition    0
GridPosition          0
Status                0
Points                0
LapTime               0
LapNumber             0
Stint                 0
Sector1Time           0
Sector2Time           0
Sector3Time           0
SpeedI1               0
SpeedI2               0
Compound              0
TyreLife              0
FreshTyre             0
Position_y            0
NameEncoder           0
dtype: int64

In [103]:
df

,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y,NameEncoder
0,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0.070413,1.0,1.0,-0.062816,1.197880,-0.532516,-0.436472,0.027647,3,-0.900151,0,1.0,13
1,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0.058025,2.0,1.0,0.413454,1.256385,-0.519926,-0.490961,-0.021169,3,-0.816672,0,1.0,13
2,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0.063755,3.0,1.0,0.426054,1.301560,-0.511492,-0.518206,-0.143208,3,-0.733194,0,1.0,13
3,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0.062426,4.0,1.0,0.415731,1.290600,-0.507091,-0.463717,0.100871,3,-0.649715,0,1.0,13
4,2,2024-03-02 15:00:00,Max Verstappen,1.0,1,1.0,1,26.0,0.069021,5.0,1.0,0.464919,1.314890,-0.502447,-0.518206,-0.094392,3,-0.566236,0,1.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191133,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,0.125421,29.0,2.0,2.992176,-0.440264,-0.538994,0.789534,-0.338472,2,-1.150587,1,19.0,17
191134,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,-0.103098,30.0,2.0,0.232946,-0.458778,-0.529827,0.707800,-0.289656,2,-1.067108,1,17.0,17
191135,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,-0.104428,31.0,2.0,0.237045,-0.463962,-0.541805,0.735045,-0.216432,2,-0.983629,1,16.0,17
191136,7,2024-07-21 13:00:00,Pierre Gasly,20.0,0,0.0,0,0.0,-0.102622,32.0,2.0,0.221560,-0.444708,-0.527626,-0.226092,-0.192024,2,-0.900151,1,15.0,17


In [104]:
df["Session5DateUtc"] = pd.to_datetime(df["Session5DateUtc"])

C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\1065486981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Session5DateUtc"] = pd.to_datetime(df["Session5DateUtc"])


In [105]:
df["Session5DateUtc"] = df["Session5DateUtc"].apply(lambda x: 1 if 6 <= x.hour < 18 else 0)

C:\Users\Caio\AppData\Local\Temp\ipykernel_20568\4103156297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Session5DateUtc"] = df["Session5DateUtc"].apply(lambda x: 1 if 6 <= x.hour < 18 else 0)


In [106]:
df.to_csv("dataframe_for_neuralnetwork.csv")

In [5]:
df = pd.read_csv("dataframe_for_neuralnetwork.csv")

In [3]:
df.head()

,Unnamed: 0,EventName,Session5DateUtc,FullName,Position_x,ClassifiedPosition,GridPosition,Status,Points,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,Compound,TyreLife,FreshTyre,Position_y
0,0,2,1,13,1.0,1.0,1.0,1,26.0,0.070413,1.0,1.0,-0.062816,1.197880,-0.532516,-0.436472,0.027647,3,-0.900151,0,1.0
1,1,2,1,13,1.0,1.0,1.0,1,26.0,0.058025,2.0,1.0,0.413454,1.256385,-0.519926,-0.490961,-0.021169,3,-0.816672,0,1.0
2,2,2,1,13,1.0,1.0,1.0,1,26.0,0.063755,3.0,1.0,0.426054,1.301560,-0.511492,-0.518206,-0.143208,3,-0.733194,0,1.0
3,3,2,1,13,1.0,1.0,1.0,1,26.0,0.062426,4.0,1.0,0.415731,1.290600,-0.507091,-0.463717,0.100871,3,-0.649715,0,1.0
4,4,2,1,13,1.0,1.0,1.0,1,26.0,0.069021,5.0,1.0,0.464919,1.314890,-0.502447,-0.518206,-0.094392,3,-0.566236,0,1.0


In [ ]:
def get_telemetry_by_driver(year, gp, driver):
    telemetry = ff1.get_session(year, gp, "R")
    telemetry.load(weather=False, messages=False, livedata=False)
    start_date = telemetry.session_info["StartDate"]
    start_date_timestamp = pd.Timestamp(start_date)
    telemetry = telemetry.laps.pick_driver(driver).get_telemetry()
    telemetry = telemetry.drop(columns=["Date", "DriverAhead", "DistanceToDriverAhead", "RPM", "Source", "Distance", "RelativeDistance", "Status", "Brake", "X", "Y", "Z"])
    telemetry["Time"] = start_date_timestamp + pd.to_timedelta(telemetry["Time"])
    telemetry.set_index('Time', inplace=True)
    resampled_data = telemetry.resample('60s').agg({
    'Speed': 'mean',
    'Throttle': 'mean',
    }).reset_index()
    return resampled_data

In [ ]:
telemetry_dataframe = []
#past_events["RoundNumber"].max() + 1
for race in range(1, 5):
    session = ff1.get_session(season, race, "R")
    session.load(weather=False, messages=False, livedata=False)
    drivers = session.results["Abbreviation"].values
    for driver in drivers:
        telemetry_data = get_telemetry_by_driver(season, race, driver)
        telemetry_data["Driver"] = driver
        telemetry_dataframe.append(telemetry_data)
            
df_telemetry_concat = pd.concat(telemetry_dataframe, axis=0, ignore_index=True)

In [ ]:
def get_weather_data(year, gp):
    weather = ff1.get_session(year, gp, "R")
    weather.load(telemetry=False, messages=False, livedata=False)
    df_weather = weather.laps.get_weather_data()
    df_weather = df_weather.drop(columns=["Humidity", "Pressure", "WindDirection", "WindSpeed"])
    return df_weather